In [37]:
import numpy
import os
import pandas as pd
from pathlib import Path

In [38]:
path_to_dir_ = Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/final3")

# # name_ = "viral"
names_ = ["viral", "human", "archea", 
          "bacteria_30", "bacteria_58", "bacteria_88", "bacteria_125"]
# # final_ = "general_results_viral500m.tsv"
finals_ = ["general_results_viral500m.tsv", 
           "general_results_human9g.tsv", 
           "general_results_archea1.4g.tsv",
           "general_results_bacteria30g.tsv",
           "general_results_bacteria58g.tsv",
           "general_results_bacteria88g.tsv",
           "general_results_bacteria125g.tsv",
           "dummy"]
path_finals = [Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_viral500m.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_human9g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_archea1.4g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria30g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria58g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria88g.tsv"),
               Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria125g.tsv")]
# path_final = Path("/Users/manuez42/Documents/results_fonda/general_results_viral500m.tsv")    

In [39]:
def convert_size(size):
    size = str(size)
    if 'G' in size:
        return float(''.join(ctp(c) for c in size if (c.isdigit() or c == ',' or c == '.'))) * 1024
    elif 'M' in size:
        return float(''.join(ctp(c) for c in size if (c.isdigit() or c == ',' or c == '.')))
    else:
        return float(''.join(ctp(c) for c in size if (c.isdigit() or c == ',' or c == '.')))

def ctp(c):
    if c == ',':
        c = '.'
    return c

def prepare_data(path_to_dir, name, final, path_final):
    filenames = filter(lambda x: (name in x) and (x != final), os.listdir(path_to_dir))
    file_paths = map((lambda x: path_to_dir / x), filenames)
    data = pd.read_csv(path_final, sep='\t')
    pd.set_option('display.max_rows', None)
    # print(data)
    
    for file in file_paths:
    # file = file_paths[0]
    # if file:
        # print(file)
        df = pd.read_csv(file)
        df = df.drop("bin_id", axis=1)
        df = df.dropna(how='any')
        
        df = df.rename(columns={"Data Size in mb": "Data Size"})
        df = df.rename(columns={"Nodes": "Node Name"})
        
        df["Data Size"] = df["Data Size"].apply(convert_size)        
        df["Node Name"] = df["Node Name"].str.replace('cmp', '').astype(int)

        data = pd.concat([data, df], axis=0)
        
    data.to_csv(path_final, index=False, sep='\t')     

In [40]:
# for i in range(0,7):
#     prepare_data(path_to_dir_, names_[i], finals_[i], path_finals[i])

In [41]:
# scp -r manuez42@allegro.imp.fu-berlin.de:/data/scratch/manuez42/A2-job-granularity/MG-HIBF/final2/ /Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/
# scp -r manuez42@allegro.imp.fu-berlin.de:/data/scratch/manuez42/A2-job-granularity/MG-HIBF/final3/ /Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/
# scp -r manuez42@allegro.imp.fu-berlin.de:/data/scratch/manuez42/A2-job-granularity/MG-HIBF/final4/ /Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/

"""
at least 10 per node 

record per nodes
"""




'\nat least 10 per node \n\nrecord per nodes\n'

In [42]:
def clean_data(path_to_f, path_to_c, finalname, threshold):
    data = pd.read_csv(path_to_f, sep='\t')
    
    #create stat
    stat = data['Node Name'].value_counts()
    stat = pd.DataFrame({'Node_Name': stat.index, 
                         'number_of_datapoints': stat.values})
    stat['dropped'] = 0
    stat['threshold'] = threshold
    stat['dataset'] = finalname

    stat = stat.sort_values(by='Node_Name')

    #drop unnecessary nodes and update stat
    rows_to_keep = []
    for _, row in stat.iterrows():
        if row['number_of_datapoints'] < threshold:
            stat.loc[stat['Node_Name'] == row['Node_Name'], 'dropped'] = 1
        else:
            rows_to_keep.append(row['Node_Name'])
    
    data = data[data['Node Name'].isin(rows_to_keep)]
    
    # print(rows_to_keep)
    
    data.to_csv(path_to_c/('cleand_' + finalname), index=False, sep='\t')
    
    return stat


In [43]:
# stat_file = Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/clean/stats.tsv")
# path_to_clean = Path("/Users/manuez42/Desktop/A2-job-granularity/MG-HIBF/general_results_fonda/clean")

# stat_col = ['dataset', 'Node_Name', 'number_of_datapoints', 'threshold', 'dropped']
# stat_overall = pd.DataFrame(columns=stat_col)


# threshold = 10

# for i in range(0,7):
#     stat_single = clean_data(path_finals[i], path_to_clean, finals_[i], threshold)
#     stat_overall = pd.concat([stat_overall, stat_single], ignore_index=True)


# stat_overall.to_csv(stat_file, index=False, sep='\t')  
    

In [44]:
# datadump

# path_to_data = [Path('/data/scratch/manuez42/genomes/viral_500M/files/'),
#                 Path('/data/scratch/manuez42/genomes/archaea_1,4G/files/'),
#                 Path('/data/scratch/manuez42/genomes/human_9G/files/'),
#                 Path('/data/scratch/manuez42/genomes/bacteria_30G/files/'),
#                 Path('/data/scratch/manuez42/genomes/bacteria_58G/files/'),
#                 Path('/data/scratch/manuez42/genomes/bacteria_88G/files/'),
#                 Path('/data/scratch/manuez42/genomes/bacteria_125G/files/')]

overview_path = [Path('/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/overview/viral_overview.tsv'),
                 Path('/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/overview/archea_overview.tsv'),
                 Path('/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/overview/human_overview.tsv'),
                 Path('/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/overview/bacteria_30_overview.tsv'),
                 Path('/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/overview/bacteria_58_overview.tsv'),
                 Path('/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/overview/bacteria_88_overview.tsv'),
                 Path('/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/overview/bacteria_125_overview.tsv')]

path_finals = [Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_viral500m.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_human9g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_archea1.4g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria30g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria58g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria88g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria125g.tsv")]

path_clean = [Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/clean/cleand_general_results_viral500m.tsv"),
              Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/clean/cleand_general_results_human9g.tsv"),
              Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/clean/cleand_general_results_archea1.4g.tsv"),
              Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/clean/cleand_general_results_bacteria30g.tsv"),
              Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/clean/cleand_general_results_bacteria58g.tsv"),
              Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/clean/cleand_general_results_bacteria88g.tsv"),
              Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/clean/cleand_general_results_bacteria125g.tsv")]

new_final_xlsx = [Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/general/general_results_viral500m.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/general/general_results_human9g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/general/general_results_archea1.4g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/general/general_results_bacteria30g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/general/general_results_bacteria58g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/general/general_results_bacteria88g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/general/general_results_bacteria125g.xlsx")]

new_clean_xlsx = [Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/cleaned_general_results_viral500m.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/cleaned_general_results_human9g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/cleaned_general_results_archea1.4g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/cleaned_general_results_bacteria30g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/cleaned_general_results_bacteria58g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/cleaned_general_results_bacteria88g.xlsx"),
                  Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/cleaned_general_results_bacteria125g.xlsx")]

overview_xlsx = [Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/overview/overview_viral500m.xlsx"),
                 Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/overview/overview_results_human9g.xlsx"),
                 Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/overview/overview_results_archea1.4g.xlsx"),
                 Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/overview/overview_results_bacteria30g.xlsx"),
                 Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/overview/overview_results_bacteria58g.xlsx"),
                 Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/overview/overview_results_bacteria88g.xlsx"),
                 Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/overview/overview_results_bacteria125g.xlsx")]


stat_file = Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/clean/stats.tsv")
stat_file_xlsx = Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/stats.xlsx")



input = [overview_path, path_finals, path_clean]
output = [overview_xlsx, new_final_xlsx, new_clean_xlsx]

In [45]:
def convert_csv_to_xlsx(csv_file, output_file):
    df_csv = pd.read_csv(csv_file)

    df_csv.to_excel(index=False)

    print(f"Data has been saved to {output_file}")
    

    
def convert_tsv_to_xlsx(tsv_file, output_file):
    
    df_tsv = pd.read_csv(tsv_file, delimiter='\t')
    
    with pd.ExcelWriter(output_file) as writer:

        df_tsv.to_excel(writer)
        
    print(f"Data has been saved to {output_file}")
    
    
convert_tsv_to_xlsx(stat_file, stat_file_xlsx)

input = [overview_path, path_finals, path_clean]
output = [overview_xlsx, new_final_xlsx, new_clean_xlsx]

for i in range(0,7):
    convert_tsv_to_xlsx(overview_path[i],overview_xlsx[i] )
    convert_tsv_to_xlsx(path_finals[i], new_final_xlsx[i])
    convert_tsv_to_xlsx(path_clean[i], new_clean_xlsx[i])

Data has been saved to /Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/stats.xlsx
Data has been saved to /Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/overview/overview_viral500m.xlsx
Data has been saved to /Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/general/general_results_viral500m.xlsx
Data has been saved to /Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/cleaned_general_results_viral500m.xlsx
Data has been saved to /Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/overview/overview_results_human9g.xlsx
Data has been saved to /Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/general/general_results_human9g.xlsx
Data has been saved to /Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/xlsx/clean/cleaned_general_results_human9g.xls